In [1]:
import numpy as np
import cv2
from cv2 import dnn

In [ ]:
proto_file = 'colorization_deploy_v2.prototxt'
model_file = 'colorization_release_v2.caffemodel'
hull_pts = 'pts_in_hull.npy'

In [ ]:
img_path="caner.jpeg"

In [ ]:
net=dnn.readNetFromCaffe(proto_file, model_file)
kernel= np.load(hull_pts)

In [ ]:
img=cv2.imread(img_path)

scaled=img.astype("float32") / 255.0
lab_img= cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)

In [ ]:
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = kernel.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

In [ ]:
resized = cv2.resize(lab_img, (224, 224))
L = cv2.split(resized)[0]
# ortalama subtraction
L -= 50

In [ ]:
net.setInput(cv2.dnn.blobFromImage(L))
ab_channel = net.forward()[0, :, :, :].transpose((1, 2, 0))
ab_channel = cv2.resize(ab_channel, (img.shape[1], img.shape[0]))

In [ ]:
L = cv2.split(lab_img)[0]
colorized = np.concatenate((L[:, :, np.newaxis], ab_channel), axis=2)

In [ ]:
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized = np.clip(colorized, 0, 1)

In [ ]:
colorized = (255 * colorized).astype("uint8")

In [ ]:
img = cv2.resize(img,(640,640))
colorized = cv2.resize(colorized,(640,640))

result = cv2.hconcat([img,colorized])
cv2.imshow("Color",colorized)
cv2.waitKey(0)


-1

In [ ]:
postprocess=cv2.addWeighted(colorized,1.5, np.zeros_like(colorized),0,0)
postprocess=cv2.GaussianBlur(postprocess, (5,5),0)
kernel=np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
postprocess=cv2.filter2D(postprocess,-1, kernel)

In [ ]:
cv2.imwrite('bradcolorized.png',colorized)

True